<a href="https://colab.research.google.com/github/hemil19/nlp-projects/blob/master/News_Category_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Predict The News Category Hackathon

#Link to the Competition:
https://www.machinehack.com/hackathons/predict_the_news_category_hackathon/overview

#Leaderboard Score:
97.99% accuracy

#Model Used:
DistilBert-Uncased

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 137kB/s 
     |████████████████████████████████| 983kB 44.7MB/s 
     |████████████████████████████████| 266kB 53.7MB/s 
     |████████████████████████████████| 1.3MB 53.6MB/s 
     |████████████████████████████████| 471kB 49.6MB/s 
     |████████████████████████████████| 2.9MB 50.3MB/s 
     |████████████████████████████████| 890kB 56.8MB/s 
     |████████████████████████████████| 1.1MB 52.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.2-cp36-none-any.whl size=25272006 sha256=977ceeef1a64d8e0b3877569b3c77247f3b3971f92c62416c4513ff92480a791
  Stored in directory: /root/.cache/pip/wheels/fe/4a/b1/aff404c4e0893ca1c0b64781d0298b7ff6de94d5117a0e7d5c
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=99cc45ec8cc07081d10af60a1d4d6e64c37bc2ec9dbad54fbd896ba971602fcc
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [2]:
import ktrain
from ktrain import text as txt

In [9]:
import pandas as pd
X=pd.read_excel('/content/Data_Train.xlsx')['STORY']
y=pd.read_excel('/content/Data_Train.xlsx')['SECTION']
test=pd.read_excel('/content/Data_Test.xlsx')['STORY']

In [25]:
for i in X[:10]:
  print(i)

But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront.


Gill’s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.


But there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor.
How formidable is the opposition alliance among Congress, Jharkhand Mukti Morcha (JMM) and Jharkhand Vikas Morcha (Prajatantrik)?
Most Asian currencies were

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
MODEL_NAME = 'distilbert-base-uncased'
t = txt.Transformer(MODEL_NAME, maxlen=100, class_names=list([0,1,2,3]))
trn = t.preprocess_train(X_train.values,y_train.values)
val=t.preprocess_train(X_test.values,y_test.values)


preprocessing train...
language: en
train sequence lengths:
	mean : 107
	95percentile : 250
	99percentile : 451


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 109
	95percentile : 255
	99percentile : 484


Is Multi-Label? False


In [12]:
trn

In [6]:
model = t.get_classifier()
model

In [7]:
X_train.shape

(6102,)

In [8]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=64)
learner.fit_onecycle(5e-5, 4)
learner.validate(class_names=t.get_classes()) # class_names must be string values



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
96/96 [==============================] - 66s 688ms/step - loss: 0.7049 - accuracy: 0.7863 - val_loss: 0.1554 - val_accuracy: 0.9653
Epoch 2/4
96/96 [==============================] - 66s 683ms/step - loss: 0.1069 - accuracy: 0.9736 - val_loss: 0.1031 - val_accuracy: 0.9712
Epoch 3/4
96/96 [==============================] - 66s 687ms/step - loss: 0.0473 - accuracy: 0.9882 - val_loss: 0.0757 - val_accuracy: 0.9738
Epoch 4/4
96/96 [==============================] - 66s 685ms/step - loss: 0.0219 - accuracy: 0.9946 - val_loss: 0.0751 - val_accuracy: 0.9784
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       323
           1       0.98      0.98      0.98       549
           2       0.99      0.99      0.99       402
           3       0.95      0.98      0.97       252

    accuracy                           0.98      1526
   macro avg       0.98      0.98      0.98

array([[308,   9,   3,   3],
       [  0, 538,   1,  10],
       [  1,   2, 399,   0],
       [  2,   2,   0, 248]])

In [13]:
predictor=ktrain.get_predictor(learner.model,preproc=t)

In [15]:
predictions=predictor.predict(test.values)

In [17]:
len(predictions),len(test)

(2748, 2748)

In [18]:
submission=pd.read_excel('/content/Sample_submission.xlsx')
submission.head()

,SECTION
0,3
1,3
2,3
3,3
4,3


In [20]:
submission['SECTION']=predictions

In [23]:
submission.to_csv('news_category_02.csv',index=False)